# ANN Self-Driving Car Attempt 3

UPDATES FROM LAST VERSION:
    - The last version was incorrectly calculating the batch sizes.
    - In this version we may also put the data-preprocessing code in a seperate class
    - Everything else stays the same
    
OOP FOR COURSEWORK:
    - Start off with taking all of the data-preprocessing steps and putting into seperate class
    - Start with an class called Data-preprocessing with main code and methods
    - Then have inhertied classes which adopt the methods and attributes but override the labeling
        - The labeling overrides the superclass labelling methods to mark the correct labels

INFO ABOUT MODEL:
    - Import all images, left, right and forward
    - Resize images to 18x22 (396 pixels)
    - Convert images to greyscale and place in new array
        - This step could be simplified by converting on the Pi, but this could slow down image recieving times
    - Reshape the array to (3000, 396)
    - Create labels for the images in parallel array form
    - Merge all three image arrays
    - Shuffle all three image and label array keeping the correct parallel structure
    - Init all tensorflow variables
    - Start interactive session
    - Train
    - Once trained and saved - you have your model!!!!

In [1]:
import tensorflow as tf                    #Machine Learning Library
import matplotlib.pyplot as plt            #Display the images for verification
import numpy as np                         #Mathematical and arrays library
from scipy import misc                     #Image Importing module
from skimage.transform import resize       #Resizing image module
from sklearn.utils import shuffle          #Shuffle in random order module
import glob                                #Iterating through all pictures library

## Greyscale Conversion Method:

In [2]:
def weightedAverage(pixel):
    return 0.299*pixel[0] + 0.587*pixel[1] + 0.114*pixel[2]

## Forward Image Array - Data Preprocessing
    - Load the path and init all variables that will be needed
    - For each image in the path that contains the .png extension
        - Resize the image to 18,22
        - Store in the array

In [3]:
# LOADING IN...
f_images = []
img_count = 0
path = "/Volumes/TRANSCEND/SDC/Jupyter Notebook Examples/cardataset/training_set/Forward/"
for filename in glob.glob(path+'*.png'):
    img_count += 1
    image = misc.imread(path+'trainframe ('+str(img_count)+').png')
    image_resized = resize(image, (18,22), mode='reflect')
    f_images.append(image_resized)

### Greyscale Conversion:
    - Init a empty numpy array of dimensions 3000, 18, 22
    - Convert each pixel to greyscale
        - OpenCV may be better at doing this - when in production, use OpenCV

In [5]:
# GREYSCALE CONVERSION...
f_grey = np.zeros((3000, image_resized.shape[0], image_resized.shape[1]))

for image_num in range(len(f_images)):
    for rownum in range(len(f_images[image_num])):
        for colnum in range(len(f_images[image_num][rownum])):
            f_grey[image_num][rownum][colnum] = weightedAverage(f_images[image_num][rownum][colnum])
    #Output Logging
    if image_num % 100 == 0:
        print("Finished converting image #", (image_num))

('Finished converting image #', 0)
('Finished converting image #', 100)
('Finished converting image #', 200)
('Finished converting image #', 300)
('Finished converting image #', 400)
('Finished converting image #', 500)
('Finished converting image #', 600)
('Finished converting image #', 700)
('Finished converting image #', 800)
('Finished converting image #', 900)
('Finished converting image #', 1000)
('Finished converting image #', 1100)
('Finished converting image #', 1200)
('Finished converting image #', 1300)
('Finished converting image #', 1400)
('Finished converting image #', 1500)
('Finished converting image #', 1600)
('Finished converting image #', 1700)
('Finished converting image #', 1800)
('Finished converting image #', 1900)
('Finished converting image #', 2000)
('Finished converting image #', 2100)
('Finished converting image #', 2200)
('Finished converting image #', 2300)
('Finished converting image #', 2400)
('Finished converting image #', 2500)
('Finished converting im

In [8]:
# MAKEING SURE THE SHAPE IS CORRECT
f_grey.shape

(3000, 18, 22)

### Reshape the greyscale images array and flatten the images to 396 pixels

In [9]:
f_images = f_grey.reshape(3000, 396)

In [11]:
# MAKEING SURE THE SHAPE CONVERSION WORKED CORRECTLY
f_images.shape

(3000, 396)

### Create the matching labels for the images
     - Init empty numpy array of 3000 by 3
     - Set the forward labels

In [12]:
# LABELS ARRAY
#Creating Labels
f_labels = np.zeros(shape=(3000,3))

#Adding 3000 forward labels [1,0,0]
for i in range(3000):
    f_labels[i][0] = float(1)

In [13]:
# CHECK OUT THE NEW LABELS
f_labels

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]])

### Shuffling time
    - First check the images
    - Shuffle
    - Recheck array to see if they have changed up

In [14]:
# CHECK THE ARRAY
f_images

array([[ 0.08285294,  0.08651373,  0.09729804, ...,  0.13449118,
         0.30126471,  0.28148725],
       [ 0.08309314,  0.08551176,  0.09758039, ...,  0.13474118,
         0.29687941,  0.2837451 ],
       [ 0.08174608,  0.08674314,  0.09856078, ...,  0.13454314,
         0.30154216,  0.2834098 ],
       ..., 
       [ 0.20243039,  0.14286569,  0.24856961, ...,  0.59031667,
         0.58096275,  0.81084804],
       [ 0.21131373,  0.14252843,  0.27131275, ...,  0.5731598 ,
         0.5833549 ,  0.82438824],
       [ 0.1865    ,  0.12876176,  0.25608725, ...,  0.56199118,
         0.58603725,  0.85044412]])

In [15]:
# SHUFFLE THEM
f_images, f_labels = shuffle(f_images, f_labels, random_state = 4000)

In [16]:
# CHECK AGAIN... - if they are different they will look different to the first time we checked
f_images

array([[ 0.20488431,  0.31913235,  0.40438627, ...,  0.93290392,
         0.91115392,  0.90059118],
       [ 0.16452059,  0.23098333,  0.24517451, ...,  0.45037843,
         0.64341667,  0.86788725],
       [ 0.52245294,  0.53974706,  0.56372451, ...,  0.88922059,
         0.87743824,  0.86189804],
       ..., 
       [ 0.12021863,  0.10482941,  0.12077941, ...,  0.52446667,
         0.47657353,  0.7271    ],
       [ 0.55091176,  0.57213137,  0.62195784, ...,  0.89393235,
         0.88511961,  0.87136176],
       [ 0.06414314,  0.05195686,  0.09219804, ...,  0.61300686,
         0.5555402 ,  0.49911863]])

## Left Image Array - Data Preprocessing
    - Load the path and init all variables that will be needed
    - For each image in the path that contains the .png extension
        - Resize the image to 18,22
        - Store in the array

In [17]:
# LOADING IN...
l_images = []
img_count = 0
path = "/Volumes/TRANSCEND/SDC/Jupyter Notebook Examples/cardataset/training_set/Left/"
for filename in glob.glob(path+'*.png'):
    img_count += 1
    image = misc.imread(path+'trainframe ('+str(img_count)+').png')
    image_resized = resize(image, (18,22), mode='reflect')
    l_images.append(image_resized)

### Greyscale Conversion:
    - Init a empty numpy array of dimensions 3000, 18, 22
    - Convert each pixel to greyscale
        - OpenCV may be better at doing this - when in production, use OpenCV

In [18]:
# GREYSCALE CONVERSION...
l_grey = np.zeros((3000, image_resized.shape[0], image_resized.shape[1]))

for image_num in range(len(l_images)):
    for rownum in range(len(l_images[image_num])):
        for colnum in range(len(l_images[image_num][rownum])):
            l_grey[image_num][rownum][colnum] = weightedAverage(l_images[image_num][rownum][colnum])
    #Output Logging
    if image_num % 100 == 0:
        print("Finished converting image #", (image_num))

('Finished converting image #', 0)
('Finished converting image #', 100)
('Finished converting image #', 200)
('Finished converting image #', 300)
('Finished converting image #', 400)
('Finished converting image #', 500)
('Finished converting image #', 600)
('Finished converting image #', 700)
('Finished converting image #', 800)
('Finished converting image #', 900)
('Finished converting image #', 1000)
('Finished converting image #', 1100)
('Finished converting image #', 1200)
('Finished converting image #', 1300)
('Finished converting image #', 1400)
('Finished converting image #', 1500)
('Finished converting image #', 1600)
('Finished converting image #', 1700)
('Finished converting image #', 1800)
('Finished converting image #', 1900)
('Finished converting image #', 2000)
('Finished converting image #', 2100)
('Finished converting image #', 2200)
('Finished converting image #', 2300)
('Finished converting image #', 2400)
('Finished converting image #', 2500)
('Finished converting im

In [19]:
# MAKEING SURE THE SHAPE IS CORRECT
l_grey.shape

(3000, 18, 22)

### Reshape the greyscale images array and flatten the images to 396 pixels

In [22]:
l_images = l_grey.reshape(3000, 396)

In [23]:
# MAKEING SURE THE SHAPE CONVERSION WORKED CORRECTLY
l_images.shape

(3000, 396)

### Create the matching labels for the images
     - Init empty numpy array of 3000 by 3
     - Set the left labels

In [ ]:
# LABELS ARRAY
#Creating Labels
l_labels = np.zeros(shape=(3000,3))

#Adding 3000 forward labels [1,0,0]
for i in range(3000):
    l_labels[i][1] = float(1)

In [ ]:
# CHECK OUT THE NEW LABELS
l_labels

### Shuffling time
    - First check the images
    - Shuffle
    - Recheck array to see if they have changed up

In [ ]:
# CHECK THE ARRAY
l_images

In [ ]:
# SHUFFLE THEM
l_images, l_labels = shuffle(l_images, l_labels, random_state = 4000)

In [ ]:
# CHECK AGAIN... - if they are different they will look different to the first time we checked
l_images

## Right Image Array - Data Preprocessing
    - Load the path and init all variables that will be needed
    - For each image in the path that contains the .png extension
        - Resize the image to 18,22
        - Store in the array

In [ ]:
# LOADING IN...
r_images = []
img_count = 0
path = "/Volumes/TRANSCEND/SDC/Jupyter Notebook Examples/cardataset/training_set/Right/"
for filename in glob.glob(path+'*.png'):
    img_count += 1
    image = misc.imread(path+'trainframe ('+str(img_count)+').png')
    image_resized = resize(image, (18,22), mode='reflect')
    r_images.append(image_resized)

### Greyscale Conversion:
    - Init a empty numpy array of dimensions 3000, 18, 22
    - Convert each pixel to greyscale
        - OpenCV may be better at doing this - when in production, use OpenCV

In [ ]:
# GREYSCALE CONVERSION...
r_grey = np.zeros((3000, image_resized.shape[0], image_resized.shape[1]))

for image_num in range(len(r_images)):
    for rownum in range(len(r_images[image_num])):
        for colnum in range(len(r_images[image_num][rownum])):
            r_grey[image_num][rownum][colnum] = weightedAverage(r_images[image_num][rownum][colnum])
    #Output Logging
    if image_num % 100 == 0:
        print("Finished converting image #", (image_num))

In [ ]:
# MAKEING SURE THE SHAPE IS CORRECT
r_grey.shape

### Reshape the greyscale images array and flatten the images to 396 pixels

In [ ]:
r_images = r_grey.reshape(3000, 396)

In [ ]:
# MAKEING SURE THE SHAPE CONVERSION WORKED CORRECTLY
r_images.shape

### Create the matching labels for the images
     - Init empty numpy array of 3000 by 3
     - Set the left labels

In [ ]:
# LABELS ARRAY
#Creating Labels
r_labels = np.zeros(shape=(3000,3))

#Adding 3000 forward labels [1,0,0]
for i in range(3000):
    r_labels[i][1] = float(1)

In [ ]:
# CHECK OUT THE NEW LABELS
r_labels

### Shuffling time
    - First check the images
    - Shuffle
    - Recheck array to see if they have changed up

In [ ]:
# CHECK THE ARRAY
r_images

In [ ]:
# SHUFFLE THEM
r_images, r_labels = shuffle(r_images, r_labels, random_state = 4000)

In [ ]:
# CHECK AGAIN... - if they are different they will look different to the first time we checked
r_images

## Merging and Shuffling All Arrays to create the dataset